In [7]:
import re
import time
from urllib.parse import quote
from bs4 import BeautifulSoup
from selenium import webdriver

def search_web(keyword):
	options = webdriver.ChromeOptions()
	options.add_argument('headless')
	options.add_experimental_option('excludeSwitches', ['enable-logging'])
	driver = webdriver.Chrome(options=options)
	driver.get(quote("https://cn.bing.com/search?q="+str(keyword),safe='/:?=.'))
	for i in range(0, 20000, 350):
		time.sleep(0.02)
		driver.execute_script('window.scrollTo(0, %s)' % i)
	html = driver.execute_script("return document.documentElement.outerHTML")
	driver.close()
	soup = BeautifulSoup(html, 'html.parser')
	item_list = soup.find_all(class_='b_algo')
	relist = []
	for items in item_list:
		item_prelist = items.find('h2')
		item_title = re.sub(r'(<[^>]+>|\s)','',str(item_prelist))
		href_s = item_prelist.find("a", href=True)
		href = href_s["href"]
		relist.append([item_title, href])
	item_list = soup.find_all(class_ ='ans_nws ans_nws_fdbk')
	for items in item_list:
		for i in range(1,10):
			item_prelist = items.find(class_ = f"nws_cwrp nws_itm_cjk item{i}", url=True, titletext=True)
			if item_prelist is not None:
				url = item_prelist["url"].replace('\ue000','').replace('\ue001','')
				title = item_prelist["titletext"]
				relist.append([title, url])
	return relist

In [8]:
rlist = search_web("重庆最长的电梯是哪个？")
rlist

[['重庆「皇冠大扶梯」是一个什么景点，有什么吸引人的…', 'https://www.zhihu.com/question/309820909'],
 ['国内最长的扶梯，全长112米，就在这座最近很火的网…', 'https://www.sohu.com/a/255836913_481035'],
 ['厉害了重庆除了超长步梯还有全国最长的电梯', 'https://www.sohu.com/a/165898534_99908704'],
 ['重庆两路口皇冠大扶梯攻略,两路口皇冠大扶梯门票_地址,两...',
  'https://www.mafengwo.cn/poi/6005643.html'],
 ['重庆最长的电梯在那_百度知道', 'https://zhidao.baidu.com/question/492952932.html'],
 ['重庆最长的扶梯在哪里-本地宝', 'http://wenda.bendibao.com/life/2021319/158970.shtm'],
 ['中国“最长”扶梯：全长112米，花两块钱就可以坐一次，就在重庆',
  'https://baijiahao.baidu.com/s?id=1676774318545512179'],
 ['全国地铁站第一深，重庆这个站台深埋地下相当于31层…', 'https://zhuanlan.zhihu.com/p/48115737'],
 ['凯旋路电梯_百度百科',
  'https://baike.baidu.com/item/%E5%87%AF%E6%97%8B%E8%B7%AF%E7%94%B5%E6%A2%AF/14313989'],
 ['重庆电梯数量超21万台，重庆需要这么多电梯吗？哪一台的...',
  'https://www.zhihu.com/question/486518855']]

In [31]:
import requests
import json

def redirect_url(url):
	headers = {
		"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36",
	}
	r = requests.get(url, headers=headers, allow_redirects=False)  # 不允许重定向
	if r.status_code == 302:
		real_url = r.headers.get('Location')
	else:
		real_url = re.findall("URL='(.*?)'", r.text)[0]
	return real_url

def search_baike(keyword):
	headers = {
		"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44",
	}
	url = quote("https://www.sogou.com/web?query="+str(keyword),safe='/:?=.')
	r = requests.get(url, headers=headers)
	html = r.text
	soup = BeautifulSoup(html, 'html.parser')
	item_list = soup.find_all(class_='struct201102')
	relist = []
	for items in item_list:
		item_prelist = items.find(class_ = "vr-title")
		item_title = re.sub(r'(<[^>]+>|\s)','',str(item_prelist))
		href_s = item_prelist.find(class_ = "", href=True)
		href = href_s["href"]
		if href[0] == "/":
			href_f = redirect_url("https://www.sogou.com"+href)
		else:
			href_f = href
		relist.append([item_title, href_f])
	for items in relist:
		if "baike.sogou.com" in items[1]:
			r = requests.get(items[1], headers=headers)
			html_s = r.text
			soup_s = BeautifulSoup(html_s, 'html.parser')
			item_list_s = soup_s.find(class_='lemma_name')
			item_title = re.sub(r'(<[^>]+>|\s)','',str(item_list_s))
			item_title = item_title.replace('编辑词条','')
			answer_sogou = soup_s.find(class_='abstract')
			answer_sogou = re.sub(r'(<[^>]+>|\s)','',str(answer_sogou))
			r = requests.get("https://baike.baidu.com/api/openapi/BaikeLemmaCardApi?scope=103&format=json&appid=379020&bk_key="+item_title+"&bk_length=600", headers=headers)
			resp = r.text.encode('utf-8').decode('gbk')
			resp_json = json.loads(resp)
			answer_baidu = resp_json.get('abstract')
			refer = resp_json.get('url')
			return [[items[1], refer], [answer_sogou, answer_baidu]]
		else:
			r = requests.get("https://baike.baidu.com/api/openapi/BaikeLemmaCardApi?scope=103&format=json&appid=379020&bk_key="+url+"&bk_length=600", headers=headers)
			resp = r.text.encode('utf-8').decode('gbk')
			resp_json = json.loads(resp)
			answer = resp_json.get('abstract')
			refer = resp_json.get('url')
			return [[refer], [answer]]

search_baike("皇冠大扶梯是什么？")

[['http://baike.sogou.com/v8072393.htm?fromTitle=%E7%9A%87%E5%86%A0%E5%A4%A7%E6%89%B6%E6%A2%AF',
  None],
 ['皇冠大扶梯，是重庆的一个路口大扶梯，连接了两路口和重庆站（又名菜园坝火车站），是重庆特色交通之一[1]。单程票价2元。该扶梯1993年2月动工，1996年2月18日建成运营，全长112米，宽1.3米，提升高度52.7米，倾斜度为30度，每秒运行0.75米，全程运行2分30秒，由上、下梯和备用梯共三台扶梯组成。每台最大载客能力为13000人次/小时，是亚洲第二长的一级提升坡地大扶梯。',
  None]]

In [6]:
import requests

NEWS_API_KEY = "3fc1f6647e6b4aaa9fc7aec282a27ea4"

def search_news(
    query: str,
    news_api_key: str = NEWS_API_KEY,
    num_articles: int = 10,
    from_datetime: str = "2023-07-01",
) -> dict:
    # https://newsapi.org/v2/everything?q=Apple&from=2023-07-29&sortBy=popularity
    response = requests.get(
        "https://newsapi.org/v2/everything",
        params={
            "q": query,
            "apiKey": news_api_key,
            "pageSize": num_articles,
            "sortBy": "popularity", # popularity / relevancy / publishedAt
            "from": from_datetime,
        },
    )

    return response.json()

search_news("LLaMA2 training")

{'status': 'ok',
 'totalResults': 24,
 'articles': [{'source': {'id': None, 'name': 'Alessiofanelli.com'},
   'author': None,
   'title': "LLaMA2 isn't “Open Source” – and why it doesn't matter",
   'description': 'Comments',
   'url': 'https://www.alessiofanelli.com/blog/llama2-isnt-open-source',
   'urlToImage': None,
   'publishedAt': '2023-07-21T15:49:04Z',
   'content': 'Almost a decade ago I started an open source company, and Ive since been involved in the OSS community as a founder, contributor, speaker, and investor. The internet wouldnt be what it is today if it… [+5230 chars]'},
  {'source': {'id': None, 'name': 'Github.com'},
   'author': 'karpathy',
   'title': 'Llama2.c: inference llama 2 in one file of pure C',
   'description': 'Inference Llama 2 in one file of pure C. Contribute to karpathy/llama2.c development by creating an account on GitHub.',
   'url': 'https://github.com/karpathy/llama2.c',
   'urlToImage': 'https://opengraph.githubassets.com/851b26341ffc8daf4c41f